In [13]:
import pandas as pd
model_data_finally=pd.read_csv(f'data/finally.csv')

In [14]:
import ast

# content에 있는 문자열을 실제 리스트로 변환하는 함수
def convert_to_list(content):
    if isinstance(content, str):
        try:
            # 문자열을 리스트 형태로 변환
            content = ast.literal_eval(content)
        except:
            # 변환에 실패하면 빈 리스트로 반환
            content = []
    return content

# merge_contents 함수는 이제 중첩된 리스트를 하나로 합침
def merge_contents(contents):
    merged = []
    for content in contents:
        content = convert_to_list(content)  # 각 content가 문자열이라면 실제 리스트로 변환
        if content:  # content가 비어 있지 않으면
            merged.extend(content)  # content의 요소들을 모두 합침
    return merged

# 특정 컬럼(tag_list)에만 적용하여 변경하는 코드
model_data_finally['tags'] = model_data_finally['tags'].apply(lambda x: merge_contents([x]))
model_data_finally['description'] = model_data_finally['description'].apply(lambda x: merge_contents([x]))
model_data_finally['address'] = model_data_finally['address'].apply(lambda x: merge_contents([x]))
model_data_finally['content'] = model_data_finally['content'].apply(lambda x: merge_contents([x]))

In [15]:
import ast

# content에 있는 문자열을 실제 리스트로 변환하는 함수
def convert_to_list(content):
    if isinstance(content, str):
        try:
            # 문자열을 리스트 형태로 변환
            content = ast.literal_eval(content)
        except:
            # 변환에 실패하면 빈 리스트로 반환
            content = []
    return content

# merge_contents 함수는 이제 중첩된 리스트를 하나로 합침
def merge_contents(contents):
    merged = []
    for content in contents:
        content = convert_to_list(content)  # 각 content가 문자열이라면 실제 리스트로 변환
        if content:  # content가 비어 있지 않으면
            merged.extend(content)  # content의 요소들을 모두 합침
    return merged


# 'place_id' 기준으로 그룹화하고, 'content'는 merge_contents로 처리
model_data_finally = model_data_finally.groupby('place_id').agg({
    'place_name': 'first',  # 첫 번째 항목 사용
    'tags': 'first',     # 첫 번째 항목 사용
    'description': 'first',  # 첫 번째 항목 사용
    'content': merge_contents  # content는 합쳐서 하나의 리스트로 만듦
}).reset_index()

In [16]:
model_data_finally['features']= model_data_finally['tags']+ model_data_finally['description']+  model_data_finally['content']
model_data_finally

,place_id,place_name,tags,description,content,features
0,78910,Gyeongbokgung Palace,[],"[Gyeongbokgung, Palace, is, located, in, the, ...","[favorite, South, Korea, worth, visit, btw, al...","[Gyeongbokgung, Palace, is, located, in, the, ..."
1,78911,Myeong Dong Cathedral,"[Churches, and, Cathedrals, Nighttime, sightse...","[Myeongdong, Cathedral, was, built, in, It, is...","[Myeongdong, Cathedral, history, year, cafe, o...","[Churches, and, Cathedrals, Nighttime, sightse..."
2,78914,Olympic Park,[],"[The, Olympic, Park, not, only, has, six, outd...","[Olympic, park, really, big, different, statio...","[The, Olympic, Park, not, only, has, six, outd..."
3,78915,63 Building,"[Wax, Museums, Modern, Architecture, Observati...","[City, is, a, skyscraper, on, Seoul, Yeouido, ...","[Scenery, nice, u, enjoyed, art, exhibition, W...","[Wax, Museums, Modern, Architecture, Observati..."
4,78916,Namsan Mountain Park,"[City, Parks, Nighttime, sightseeing]","[Namsan, Park, is, a, landmark, in, Seoul, and...","[one, Seoul, rsquo, best, view, superb, cable,...","[City, Parks, Nighttime, sightseeing, Namsan, ..."
...,...,...,...,...,...,...
946,149396390,Kang Junseok - MELANGE,[],"[Loved, by, the, public, for, reminiscent, of,...",[],"[Loved, by, the, public, for, reminiscent, of,..."
947,149396515,Welcome Daehakro Festival,[],"[The, Welcome, Daehakno, Festival, which, has,...",[],"[The, Welcome, Daehakno, Festival, which, has,..."
948,149541140,damien rice 2025 tour in Seoul,[],"[No, Description]",[],"[No, Description]"
949,149542333,Yongsun Kim <Unknown area>,[],"[The, feeling, of, brushing, every, day, is, l...",[],"[The, feeling, of, brushing, every, day, is, l..."


In [26]:
import pandas as pd
from gensim.models import Word2Vec
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# 각 문장을 단어 리스트로 변환
sentences = model_data_finally['features']


# Word2Vec 모델 학습
word2vec_model = Word2Vec(sentences, vector_size=1000, window=7, min_count=2, workers=4)
# vector_size=100: 각 단어를 100차원의 벡터로 표현합니다. (벡터의 차원이 높을수록 단어 의미를 더 잘 포착할 수 있습니다.)
# window=5: 학습 시 각 단어의 앞뒤로 5개 단어를 고려합니다. (문맥을 파악하는 범위)
# min_count=1: 최소 1회 이상 등장한 단어만 학습에 포함합니다. (드물게 등장하는 단어도 포함)
# workers=4: 모델 학습에 사용할 CPU 코어 수입니다.


# Step 2: 각 문장에 대한 평균 벡터 계산
def get_average_word2vec_vector(words):
    word_vectors = [word2vec_model.wv[word] for word in words if word in word2vec_model.wv]
    if len(word_vectors) == 0:
        return np.zeros(word2vec_model.vector_size)
    return np.mean(word_vectors, axis=0)

# 모든 문장에 대해 평균 벡터 계산
vectors = np.array([get_average_word2vec_vector(feature) for feature in model_data_finally['features']])

In [28]:
# 특정 관광지와 유사한 관광지 찾기 함수 (place_id로 입력)
def get_similar_places_by_id(place_id):
    # 특정 관광지의 인덱스를 찾음
    place_row = model_data_finally[model_data_finally['place_id'] == place_id]
    if place_row.empty:
        raise ValueError(f"place_id '{place_id}' not found.")
    
    place_idx = place_row.index[0]
    
    # 해당 관광지와 모든 관광지의 유사도 계산
    cosine_sim = cosine_similarity([vectors[place_idx]], vectors)
    
    # 유사도 높은 관광지 찾기
    similarity_scores = list(enumerate(cosine_sim[0]))
    similarity_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)
    
    # 유사한 관광지 정보 출력 (본인 제외)
    similar_places = []
    for idx, score in similarity_scores[1:11]:  # 본인 제외 후 상위 10개
        similar_places.append({
            'place_name': model_data_finally['place_name'][idx],  # 관광지 이름
            'place_id': model_data_finally['place_id'][idx],  # 관광지 ID
            'similarity_score': score  # 유사도 점수
        })
    
    return similar_places

# 예시: 특정 place_id로 유사한 관광지 찾기
similar_places = get_similar_places_by_id(78914)  # place_id 예제
for i in similar_places:
    print(i['place_id'])

90331
31690574
58266642
95245
24652381
10559325
10794367
23583248
99870
23031760
